In [12]:
import torch.nn as nn
import torch.utils.model_zoo as model_zoo
import torch
from torch.nn import functional as F2
from PIL import Image
import torch.utils.data as data
import os
from glob import glob
import torchvision.transforms.functional as F
from torchvision import transforms, models
import random
import numpy as np
from torch.utils.data import DataLoader
from torch.utils.data.dataloader import default_collate
from torch import optim
from torch.nn import Module
import time
import torchvision
import torchvision.transforms as transforms
import matplotlib as plt

## Orginal model of the paper

A pretrain vvg19 nn with a downsampling of 8.

In [13]:
__all__ = ['vgg19']
model_urls = {
    'vgg19': 'https://download.pytorch.org/models/vgg19-dcbb9e9d.pth',
}

class VGG(nn.Module):
    def __init__(self, features):
        super(VGG, self).__init__()
        self.features = features
        self.reg_layer = nn.Sequential(
            nn.Conv2d(512, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 1, 1)
        )

    def forward(self, x):
        x = self.features(x)
        x = F2.upsample_bilinear(x, scale_factor=2)
        x = self.reg_layer(x)
        return torch.abs(x)


def make_layers(cfg, batch_norm=False):
    layers = []
    in_channels = 3
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)

cfg = {
    'E': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512]
}

def vgg19():
    """VGG 19-layer model (configuration "E")
        model pre-trained on ImageNet
    """
    model = VGG(make_layers(cfg['E']))
    model.load_state_dict(model_zoo.load_url(model_urls['vgg19']), strict=False)
    return model

## CSRNet 
Downsampling = 1

In [14]:
class CSRNet(nn.Module):
    def __init__(self, load_weights=False):
        super(CSRNet, self).__init__()
        self.seen = 0
        self.frontend_feat = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512]
        self.backend_feat  = [512, 512, 512,256,128,64]
        self.frontend = make_layers(self.frontend_feat)
        self.backend = make_layers(self.backend_feat,in_channels = 512,dilation = True)
        self.output_layer = nn.Conv2d(64, 1, kernel_size=1)
        if not load_weights:
            mod = models.vgg16(pretrained = True)
            self._initialize_weights()
            self.frontend.load_state_dict(mod.features[0:23].state_dict())
            
    def forward(self,x):
        size = x.size()
        x = self.frontend(x)
        x = self.backend(x)
        x = self.output_layer(x)
        x = F2.upsample(x, size = size[2:])
        return x
    
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.normal_(m.weight, std=0.01)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            
                
def make_layers(cfg, in_channels = 3,batch_norm=False,dilation = False):
    if dilation:
        d_rate = 2
    else:
        d_rate = 1
    layers = []
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=d_rate,dilation = d_rate)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)  

## Processing helpers

In [15]:
def random_crop(im_h, im_w, crop_h, crop_w):
    res_h = im_h - crop_h
    res_w = im_w - crop_w
    i = random.randint(0, res_h)
    j = random.randint(0, res_w)
    return i, j, crop_h, crop_w


def cal_innner_area(c_left, c_up, c_right, c_down, bbox):
    inner_left = np.maximum(c_left, bbox[:, 0])
    inner_up = np.maximum(c_up, bbox[:, 1])
    inner_right = np.minimum(c_right, bbox[:, 2])
    inner_down = np.minimum(c_down, bbox[:, 3])
    inner_area = np.maximum(inner_right-inner_left, 0.0) * np.maximum(inner_down-inner_up, 0.0)
    return inner_area



class Crowd(data.Dataset):
    def __init__(self, root_path, crop_size,
                 downsample_ratio, is_gray=False,
                 method='train'):

        self.root_path = root_path
        self.im_list = sorted(glob(os.path.join(self.root_path, '*.jpg')))
        if method not in ['train', 'val']:
            raise Exception("not implement")
        self.method = method

        self.c_size = crop_size
        self.d_ratio = downsample_ratio
        assert self.c_size % self.d_ratio == 0
        self.dc_size = self.c_size // self.d_ratio

        if is_gray:
            self.trans = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
            ])
        else:
            self.trans = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # Pour CSRNet à vérifier si besoin de modif
            ])

    def __len__(self):
        return len(self.im_list)

    def __getitem__(self, item):
        img_path = self.im_list[item]
        gd_path = img_path.replace('jpg', 'npy')
        img = Image.open(img_path).convert('RGB')
        if self.method == 'train':
            keypoints = np.load(gd_path)
            return self.train_transform(img, keypoints)
        elif self.method == 'val':
            keypoints = np.load(gd_path)
            img = self.trans(img)
            name = os.path.basename(img_path).split('.')[0]
            return img, len(keypoints), name

    def train_transform(self, img, keypoints):
        """random crop image patch and find people in it"""
        
        """
        Les keypoints correspondent aux coordonnées des têtes
        MAIS une troisième coordonnée a été calculée lors du preprocessing des données,
        elle correspont à "dis" et semble important pour calculer pas mal de choses
        """
        
        wd, ht = img.size
        st_size = min(wd, ht)
        assert st_size >= self.c_size
        assert len(keypoints) > 0
        i, j, h, w = random_crop(ht, wd, self.c_size, self.c_size)
        img = F.crop(img, i, j, h, w)
        
        nearest_dis = np.clip(keypoints[:, 2], 4.0, 128.0)
       
        points_left_up = keypoints[:, :2] - nearest_dis[:, None] / 2.0
        points_right_down = keypoints[:, :2] + nearest_dis[:, None] / 2.0
        bbox = np.concatenate((points_left_up, points_right_down), axis=1)
        inner_area = cal_innner_area(j, i, j+w, i+h, bbox)
        origin_area = nearest_dis * nearest_dis
        ratio = np.clip(1.0 * inner_area / origin_area, 0.0, 1.0)
        mask = (ratio >= 0.3)

        target = ratio[mask]
        keypoints = keypoints[mask]
        keypoints = keypoints[:, :2] - [j, i]  # change coodinate
        if len(keypoints) > 0:
            if random.random() > 0.5:
                img = F.hflip(img)
                keypoints[:, 0] = w - keypoints[:, 0]
        else:
            if random.random() > 0.5:
                img = F.hflip(img)
        return self.trans(img), torch.from_numpy(keypoints.copy()).float(), \
               torch.from_numpy(target.copy()).float(), st_size

## Computation of p(yn|xm)

In [16]:
class Post_Prob(Module):
    def __init__(self, sigma, c_size, stride, background_ratio, use_background, device):
        super(Post_Prob, self).__init__()
        assert c_size % stride == 0

        self.sigma = sigma
        self.bg_ratio = background_ratio
        self.device = device
        # coordinate is same to image space, set to constant since crop size is same
        self.cood = torch.arange(0, c_size, step=stride,
                                 dtype=torch.float32, device=device) + stride / 2
        self.cood.unsqueeze_(0)
        self.softmax = torch.nn.Softmax(dim=0)
        self.use_bg = use_background

    def forward(self, points, st_sizes):
        num_points_per_image = [len(points_per_image) for points_per_image in points]
        all_points = torch.cat(points, dim=0)

        if len(all_points) > 0:
            x = all_points[:, 0].unsqueeze_(1)
            y = all_points[:, 1].unsqueeze_(1)
            x_dis = -2 * torch.matmul(x, self.cood) + x * x + self.cood * self.cood
            y_dis = -2 * torch.matmul(y, self.cood) + y * y + self.cood * self.cood
            y_dis.unsqueeze_(2)
            x_dis.unsqueeze_(1)
            dis = y_dis + x_dis
            dis = dis.view((dis.size(0), -1))

            dis_list = torch.split(dis, num_points_per_image)
            prob_list = []
            for dis, st_size in zip(dis_list, st_sizes):
                if len(dis) > 0:
                    if self.use_bg:
                        min_dis = torch.clamp(torch.min(dis, dim=0, keepdim=True)[0], min=0.0)
                        d = st_size * self.bg_ratio
                        bg_dis = (d - torch.sqrt(min_dis))**2
                        dis = torch.cat([dis, bg_dis], 0)  # concatenate background distance to the last
                    dis = -dis / (2.0 * self.sigma ** 2)
                    prob = self.softmax(dis)
                else:
                    prob = None
                prob_list.append(prob)
        else:
            prob_list = []
            for _ in range(len(points)):
                prob_list.append(None)
        return prob_list

## Bayesian Loss definition

In [17]:
class Bay_Loss(Module):
    def __init__(self, use_background, device):
        super(Bay_Loss, self).__init__()
        self.device = device
        self.use_bg = use_background

    def forward(self, prob_list, target_list, pre_density):
        # print("problist", prob_list[0].size())
        # print("pre dense", pre_density.size())
        # print("target_list", target_list)
        loss = 0
        
        """
            - prob list semble être la listes des p(yn|xm) ie la contribution du pixel xm sur la n-ieme tête
            (les lignes de cette matrice sont de taille 4096 = 64*64)
            - pre density est la prédiction de la densité (sortie du réseau) - de taille 64x64 ici
            - target list a pour longueur le nombre de têtes - correspond aux E[cn] "réel" (le calcul reste un mystère)
            - On obtient les E[cn] estimées grâce à un produit terme à terme de prob_list et pre_density
        """
        
        
        for idx, prob in enumerate(prob_list):  # iterative through each sample
            if prob is None:  # image contains no annotation points
                pre_count = torch.sum(pre_density[idx])
                target = torch.zeros((1,), dtype=torch.float32, device=self.device)
            else:
                N = len(prob)
                if self.use_bg:
                    target = torch.zeros((N,), dtype=torch.float32, device=self.device)
                    target[:-1] = target_list[idx]
                else:
                    target = target_list[idx]
                pre_count = torch.sum(pre_density[idx].view((1, -1)) * prob, dim=1)  # flatten into vector

            loss += torch.sum(torch.abs(target - pre_count))
        loss = loss / len(prob_list)
        return loss

In [18]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = 1.0 * self.sum / self.count

    def get_avg(self):
        return self.avg

    def get_count(self):
        return self.count

# Datasets and DataLoader

In [19]:
downsample_ratio = 8 # Mettre à 8 pour le réseau du répo (à 1 pour CSRNet puisque on ne modifie pas la dim avec le réseau)
data_dir = "SHHA"
# data_dir = "/Users/VictoRambaud/dev/crowd_counting2/ProcessedData/SHHA"
crop_size = 256
is_gray = False
num_workers = 8
batch_size = 10

def train_collate(batch):
    transposed_batch = list(zip(*batch))
    # print(transposed_batch)
    images = torch.stack(transposed_batch[0], 0)
    points = transposed_batch[1]  # the number of points is not fixed, keep it as a list of tensor
    targets = transposed_batch[2]
    st_sizes = torch.FloatTensor(transposed_batch[3])
    return images, points, targets, st_sizes


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Using gpu: %s ' % torch.cuda.is_available())

datasets = {x: Crowd(os.path.join(data_dir, x),
                          crop_size,
                          downsample_ratio,
                          is_gray, x) for x in ['train', 'val']}

dataloaders = {x: DataLoader(datasets[x],
                            collate_fn=(train_collate if x == 'train' else default_collate),
                            batch_size=(batch_size if x == 'train' else 1),
                            shuffle=(True if x == 'train' else False),
                            num_workers=num_workers,
                            pin_memory=(True if x == 'train' else False))
                            for x in ['train', 'val']}

Using gpu: True 


## Model / Params

In [20]:
lr = 0.0001
weight_decay = 0.9
sigma = 0.1
use_background = False
background_ratio = 1

model = vgg19()
model.to(device)
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

post_prob = Post_Prob(sigma,
                           crop_size,
                           downsample_ratio,
                           background_ratio,
                           use_background,
                           device)
criterion = Bay_Loss(use_background, device)

In [21]:
from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter()

image = Image.open('SHHA/train/IMG_1.jpg')
trans1 = transforms.ToTensor()
img = trans1(image).to(device)
img = img.unsqueeze(0)

# write to tensorboard

writer.add_graph(model, img)
writer.close()

/opt/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:2589: UserWarning: nn.functional.upsample_bilinear is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample_bilinear is deprecated. Use nn.functional.interpolate instead.")


In [22]:
# tensorboard --logdir=runs

## Training

In [25]:
start_epoch = 0
max_epoch = 20
validation_epoch = 1
val_start = 0
save_dir = ""

def train():
    for epoch in range(start_epoch, max_epoch):
        print('-'*5 + 'Epoch {}/{}'.format(epoch, max_epoch - 1) + '-'*5)
        train_epoch(epoch)
        if epoch % validation_epoch == 0 and epoch >= val_start:
            val_epoch(epoch)
        

def train_epoch(epoch):
    epoch_loss = AverageMeter()
    epoch_mae = AverageMeter()
    epoch_mse = AverageMeter()
    epoch_start = time.time()
    model.train()  # Set model to training mode
    running_loss = 0.0
    # Iterate over data.
    for step, (inputs, points, targets, st_sizes) in enumerate(dataloaders['train']):
        inputs = inputs.to(device)
        st_sizes = st_sizes.to(device)
        gd_count = np.array([len(p) for p in points], dtype=np.float32)
        points = [p.to(device) for p in points]
        targets = [t.to(device) for t in targets]
        
        # print(inputs.size())
        # print(gd_count)
        # print(points)
        # print(targets)
        # print(st_sizes)
        
        # inputs = image size(m, 3, crop_size, crop_size)
        # gd_counts = [nb_of_head_img1, num_of_head_img2, ...]
        # point = array of positions
        # targets = ?
        

        with torch.set_grad_enabled(True):
            outputs = model(inputs)
            prob_list = post_prob(points, st_sizes)
            loss = criterion(prob_list, targets, outputs)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            

            N = inputs.size(0) # batch size
            pre_count = torch.sum(outputs.view(N, -1), dim=1).detach().cpu().numpy()
            res = pre_count - gd_count
            
            print(f'step: {step} | gd_count: {gd_count} | prediction: {pre_count} | loss: {loss}')
            
            running_loss += loss.item()
            if step % 2 == 1:    # every 1000 mini-batches...

            # ...log the running loss
                writer.add_scalar('training loss vgg',
                            running_loss / 2,
                            epoch * len(dataloaders['train']) + step)
                running_loss = 0.0
            # epoch_loss.update(loss.item(), N)
            # epoch_mse.update(np.mean(res * res), N)
            # epoch_mae.update(np.mean(abs(res)), N)

    # print('Epoch {} Train, Loss: {:.2f}, MSE: {:.2f} MAE: {:.2f}, Cost {:.1f} sec'
    #             .format(epoch, epoch_loss.get_avg(), np.sqrt(epoch_mse.get_avg()), epoch_mae.get_avg(),
    #                     time.time()-epoch_start))

def val_epoch(epoch):
    epoch_start = time.time()
    model.eval()  # Set model to evaluate mode
    epoch_res = []
    running_loss = 0.0
    best_mse = 100000000000000
    best_mae = 100000000000000

    # Iterate over data.
    for inputs, count, name in dataloaders['val']:
        inputs = inputs.to(device)
        # inputs are images with different sizes
        assert inputs.size(0) == 1, 'the batch size should equal to 1 in validation mode'
        with torch.set_grad_enabled(False):
            outputs = model(inputs)
            res = count[0].item() - torch.sum(outputs).item()
            epoch_res.append(res)
    
            
    running_loss = 0.0
    epoch_res = np.array(epoch_res)
    mse = np.sqrt(np.mean(np.square(epoch_res)))
    mae = np.mean(np.abs(epoch_res))
    
                # ...log the running loss
    writer.add_scalar('val MAE vgg',
                        mae,
                        epoch * len(dataloaders['val']))
    writer.add_scalar('val MSE vgg',
                    mse,
                    epoch * len(dataloaders['val']))
    
    print('Epoch {} Val, MSE: {:.2f} MAE: {:.2f}, Cost {:.1f} sec'
                 .format(epoch, mse, mae, time.time()-epoch_start))

    model_state_dic = model.state_dict()
    if (2.0 * mse + mae) < (2.0 * best_mse + best_mae):
        best_mse = mse
        best_mae = mae
        print("save best mse {:.2f} mae {:.2f} model epoch {}".format(best_mse,
                                                                             best_mae,
                                                                             epoch))
        torch.save(model_state_dic, os.path.join(save_dir, 'best_model_vgg.pth'))

In [26]:
train()

-----Epoch 0/19-----
step: 0 | gd_count: [ 45. 140.   2. 172.  64.  18.  68.  76.  59.  34.] | prediction: [47.201492 54.46009  67.745605 37.82522  42.88755  52.49192  45.365654
 57.99845  40.121384 59.44285 ] | loss: 59.54890823364258
step: 1 | gd_count: [  8.  42.  46.  36.   9.   6. 138. 137. 105.  53.] | prediction: [36.02681  58.389755 52.527718 15.338438 16.306213 17.222683 41.62084
 40.654545 44.01917  20.833183] | loss: 47.82880783081055
step: 2 | gd_count: [ 87. 128.  84.  71.  20. 139.  83.  26.  48.  29.] | prediction: [50.686    40.238262 37.817917 45.785328 30.189236 47.218864 55.567574
 30.30301  50.79026  36.39255 ] | loss: 39.39012908935547
step: 3 | gd_count: [18. 61. 54.  8. 46. 22. 54. 20.  0. 93.] | prediction: [ 58.316944 151.03441  111.57718   28.993057  91.076614  61.371334
  31.633179  58.8022    30.29679  153.08449 ] | loss: 49.6040153503418
step: 4 | gd_count: [ 18.  47.  30.  36.  42.  30. 180.   9.   9.  23.] | prediction: [19.549196 32.649403 20.8181   24.9

step: 13 | gd_count: [61.  7. 44. 55. 24. 56. 38. 74. 36. 27.] | prediction: [33.009186  6.487939 40.257225 38.1781    9.606043 40.20549  33.01216
 39.49661  10.817812 18.578459] | loss: 16.268089294433594
step: 14 | gd_count: [ 58. 103.  91.   2.  97. 204.  92.  73.   7.  95.] | prediction: [16.815567 54.43081  40.157677  9.700008 27.852978 66.29373  51.783287
 43.658287  6.274604 93.303215] | loss: 43.2924919128418
step: 15 | gd_count: [94. 36. 85. 40. 18. 24. 57. 11. 81. 36.] | prediction: [36.990303 14.196057 38.62406  22.134258 12.328886 14.97151  22.483286
 13.805094 55.95852  24.395   ] | loss: 20.482662200927734
step: 16 | gd_count: [  5. 510.  38. 190.  26.  57.   7.  54.   9.  81.] | prediction: [  9.335019 163.68451   21.53992   87.369675  19.848696  39.747425
  15.057012  31.372494  15.44116   43.661995] | loss: 54.908912658691406
step: 17 | gd_count: [ 44. 224.  83.   0.  47. 251.  42.  34.  18. 133.] | prediction: [ 52.758636 166.86923   52.24382   16.681894  32.238388 11

step: 1 | gd_count: [19. 40. 61. 36. 39. 40. 24. 39. 94.  8.] | prediction: [28.121485 32.279324 58.447105 28.42448  49.990265 33.14125  18.04244
 37.643303 59.46068  15.292013] | loss: 15.075759887695312
step: 2 | gd_count: [292.  48.  62.  93.   4.  58. 182.  72.  47. 110.] | prediction: [109.838455  67.60857   55.318504  97.55666    6.628502  56.78998
 155.84763   83.52777   62.18483   90.005196] | loss: 46.4315185546875
step: 3 | gd_count: [ 45.  59. 100.   9.  27.  12.  22. 228.  35.  75.] | prediction: [ 45.131958  59.610878 111.45326    7.696452  25.519997  15.561861
  17.600603 158.45549   21.251532  61.638813] | loss: 20.58091163635254
step: 4 | gd_count: [ 26. 139.  22. 114.  99. 234. 206.  23.  57.  61.] | prediction: [ 20.48516  125.00051   16.538055  84.88945   63.382477 223.05624
 217.89523   11.945097  35.21827   60.32328 ] | loss: 31.648767471313477
step: 5 | gd_count: [ 13.  12. 122.  52.  28. 366.  13.  22.  25.   0.] | prediction: [ 11.074343   16.810843  143.15811  

step: 13 | gd_count: [ 40.  97. 134.  18. 427.  12. 107.  20. 174.  12.] | prediction: [50.516125 65.76688  50.02817  45.63752  76.47659  20.806725 29.181297
 37.8542   78.3365   26.00324 ] | loss: 75.46580505371094
step: 14 | gd_count: [ 26.  52.  27.  30. 283. 118.  36. 109. 103.   5.] | prediction: [42.095978  65.0597    61.156105  19.591694  52.280228  17.917995
 47.2745    41.363743  84.34032    4.4949703] | loss: 53.8752326965332
step: 15 | gd_count: [ 42.  47. 100.  76.  29.  96.  22.  52.  27.  19.] | prediction: [33.859    23.24554  33.500145 28.698656 13.271954 70.58922  16.760408
 57.615307 13.511316 10.262478] | loss: 21.234189987182617
step: 16 | gd_count: [ 63.  84. 191.  65. 143.   9.  51.  65.  89.  49.] | prediction: [ 48.317635  43.936577  75.162544  70.07887  109.82922    5.310277
  38.77464   56.29883   11.8911    25.199785] | loss: 41.47551345825195
step: 17 | gd_count: [ 33.  21.  53.  93. 112.  64.  49.  45.  32.   5.] | prediction: [ 33.582703  35.45185   75.445

step: 1 | gd_count: [ 53.   7. 151.  74. 248. 146.  26.  24.  41.  74.] | prediction: [ 51.819008    7.9714704  90.158615   63.874187  157.06012   142.57199
  21.812302   23.899836   26.877457   73.23111  ] | loss: 29.04669189453125
step: 2 | gd_count: [ 70.  55.  32.  58.   5. 329.   6.   3.  66. 213.] | prediction: [ 66.446655  60.67163   40.61698   87.06238   12.158604 223.58356
  11.30558   17.920301  70.4306   128.99815 ] | loss: 35.39836502075195
step: 3 | gd_count: [ 68.  98.  29. 186.  45.  47.  68.  77.  33.  25.] | prediction: [ 86.24136   73.96974   16.427973 145.31711   38.181625  45.938206
 106.08855   62.14646   39.923637  25.034433] | loss: 28.346662521362305
step: 4 | gd_count: [  0.   0.  23. 199. 216.  24. 161. 172.  15.  46.] | prediction: [  6.189595    6.23643    23.036364  132.11678   173.0786     25.670797
 154.02805   181.08131    15.5499115  72.4321   ] | loss: 32.192108154296875
step: 5 | gd_count: [ 52.  31. 325.  34.  26. 204.   1. 118.  60.   7.] | predicti

...

step: 12 | gd_count: [117.  33. 101.   8.  30.  20.  70.  52.  25. 201.] | prediction: [ 85.66089   13.726579  95.126274   6.5905     9.282248  15.723199
  34.473595  30.888756  22.962059 199.47963 ] | loss: 25.6405086517334
step: 13 | gd_count: [106.  33.  17.   6.   0.  60. 134.  22. 264.  52.] | prediction: [ 56.065327   22.794872    9.858514    4.1125      1.4920177  25.827635
  73.763145   13.781689  173.74484    32.43454  ] | loss: 26.742237091064453
step: 14 | gd_count: [  0.   0.   0.  27.  58. 187.  63.   2.  39. 226.] | prediction: [  1.7255912   1.2090404   1.1959488  16.35579    33.93337   188.92978
  40.292732    2.3915424  21.541458  224.794    ] | loss: 25.676198959350586
step: 15 | gd_count: [ 59.  19.  18. 354.  71.  71.   0.  47. 121.  23.] | prediction: [ 53.04491    20.07711    14.916749  278.1274     46.626278   46.690895
   1.5529189  55.311928  106.889435   20.139091 ] | loss: 24.798574447631836
step: 16 | gd_count: [ 56.   0.  43. 122. 101.  66.  20.  21.   6.  

Epoch 14 Val, MSE: 302.49 MAE: 228.56, Cost 6.8 sec
save best mse 302.49 mae 228.56 model epoch 14
-----Epoch 15/19-----
step: 0 | gd_count: [337.  39.  96.  52. 201.  99.  44.  48.  76. 114.] | prediction: [178.62871    33.601295    2.7218914  11.688694  225.6294     36.021202
  22.43301    16.265213   31.404041   60.799397 ] | loss: 59.55498123168945
step: 1 | gd_count: [  7.   1. 208.  51.  49. 222.  19. 173. 331.  15.] | prediction: [  1.9388442   2.0561917 154.84831    33.43376    16.851067  145.87437
  11.351614  104.47233   264.8855      6.582435 ] | loss: 43.51158142089844
step: 2 | gd_count: [  6.  44. 494.  17. 115. 458. 118.  43. 140.  34.] | prediction: [  5.420043  37.592445 375.2571    15.82601   83.03565  212.58769
  87.10195   21.273106 111.29173   31.872477] | loss: 68.86016845703125
step: 3 | gd_count: [21. 94. 51. 19. 27. 26.  8. 38. 20. 62.] | prediction: [20.646107 93.735916 71.6266   19.404594 27.718548 34.19413   9.542484
 21.94976  21.324669 62.72224 ] | loss: 1

step: 11 | gd_count: [ 39.  73. 131.   8.  72. 278.  53. 106.   9.  68.] | prediction: [ 48.86666    67.33464   107.28959     3.9984488  80.673546  173.77776
  61.098587   88.480545    6.576332   69.32762  ] | loss: 30.785322189331055
step: 12 | gd_count: [ 16.  35.  98.  44.  13.   8. 170.   3.  17.   8.] | prediction: [ 27.048931   39.57807    92.58626    43.06646    18.455822    7.0463753
 132.37984     4.813898   18.281775   11.264877 ] | loss: 14.210138320922852
step: 13 | gd_count: [ 21.  57.  16. 101.  28. 101.  12.  21.  27.  30.] | prediction: [ 24.397282  53.652657  18.35064  151.95142   22.985     73.19591
   8.108151  22.414562  48.625378  40.44884 ] | loss: 19.301599502563477
step: 14 | gd_count: [448.  17.  80.   0. 143.  12.  38. 296.  84.  67.] | prediction: [321.50613    11.4677515  62.117893    2.4708002  94.43657    11.446819
  31.058823  148.06308    64.01347    50.724503 ] | loss: 42.863521575927734
step: 15 | gd_count: [ 50. 132.  35.  75.  35.  36.  21. 481. 149.

step: 23 | gd_count: [ 97.  52.  39. 252. 281.   0.  90.   5.   8.] | prediction: [ 86.11427   43.25642   31.49714  131.58063  165.37344    8.110395
  60.149166   5.376668  11.40314 ] | loss: 38.21353530883789
Epoch 17 Val, MSE: 284.92 MAE: 144.43, Cost 6.8 sec
save best mse 284.92 mae 144.43 model epoch 17
-----Epoch 18/19-----
step: 0 | gd_count: [  0. 180. 108.   8.  66.  14.  51.  70.  40.  41.] | prediction: [  2.1678019 194.78769   121.73546     9.872475   52.792213   13.009092
  37.016907   48.031906   38.708565   38.445824 ] | loss: 21.11506462097168
step: 1 | gd_count: [157.   1. 236.  88.   8.   2.   9.  41. 406.  19.] | prediction: [ 88.21001     3.4300704 184.28644    94.02672     4.20651     3.8528092
   8.037836   34.3765    231.26187    18.561722 ] | loss: 36.59150695800781
step: 2 | gd_count: [ 89.  45.  45.  25. 458. 301.  23.  55.  85.  10.] | prediction: [106.17685   36.370094  66.51525   18.97865  223.2131   333.96283
  27.47361   57.007225  82.65147    9.932459] | 

step: 10 | gd_count: [ 45.   2. 156.  19.  22. 121.   0.  26.   0.  42.] | prediction: [30.528698  6.414038 83.72128  11.198234 22.54165  44.968803  5.532557
 21.901278  6.199065 36.744453] | loss: 19.115867614746094
step: 11 | gd_count: [262.   8.  65.   3. 138.  20.  68. 240.   5. 373.] | prediction: [ 84.99544     4.8218956  45.206276    5.5766935  76.30916    20.333542
  80.8613    114.34697     5.086893  103.60603  ] | loss: 66.67695617675781
step: 12 | gd_count: [36. 21. 15. 17. 41. 35. 21. 67. 30. 26.] | prediction: [30.81688  15.772711 21.77002  17.168987 42.218018 41.202793 18.788397
 69.37594  24.027208 27.378807] | loss: 9.23518180847168
step: 13 | gd_count: [77. 86. 64. 21. 41. 15. 27. 56. 59. 42.] | prediction: [81.435776 71.4193   53.015083 20.353054 12.260988 13.929901 13.99749
 71.704735 73.991974 51.46001 ] | loss: 17.992023468017578
step: 14 | gd_count: [ 81.  50. 147.  13.  70.  46. 125.  21.  93.  33.] | prediction: [ 84.77553   39.47283  121.91572    8.36242   70.5

In [79]:
from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter('tensorboard_test')


In [80]:
writer.add_graph(model)
writer.close()

TypeError: 'NoneType' object is not iterable

In [ ]:
next(iter(dataloaders['val']))

In [50]:
import os

perm = np.random.permutation(300)

In [59]:
### ATTENTION - NE PAS TOUCHER

for i in range(300):
    if i < 60:
        path_img = os.path.join("SHHA/train", f'IMG_{perm[i]}.jpg')
        path_npy = os.path.join("SHHA/train", f'IMG_{perm[i]}.npy')
        # os.remove(path_img)
        # os.remove(path_npy)
    else:
        if i > 91:
            path_img = os.path.join("SHHA/val", f'IMG_{perm[i]}.jpg')
            path_npy = os.path.join("SHHA/val", f'IMG_{perm[i]}.npy')
            ## os.remove(path_img)
            ## os.remove(path_npy)

In [56]:
for i in range(300):
    if perm[i] == 0:
        print(i)

91


In [58]:
perm

array([  5, 229,  86, 246, 238,  22, 167, 277, 212,   6,  93, 197, 267,
       228,  45,  37, 111,  80, 241, 260,  30,  48, 262,  71, 179, 160,
       181, 146, 163, 161, 172, 278, 109, 104, 201, 155, 183, 296,  41,
       108,  14,  78, 251, 185, 194, 199, 139, 195,  98, 255, 131, 122,
        11,  21, 214,  58, 219,  65, 119, 145, 275,  82, 126,  92, 188,
        38, 112, 149,   4, 253, 143,  47,  75, 206,  73,  64,  56, 268,
       159, 274, 235, 297, 171,  25,   2,  29,  44, 285,  90, 137,  77,
         0, 207, 107, 117,  52, 168,   8, 138, 269, 299, 169,  66,  91,
       115,  42, 209, 225, 240, 153, 125, 105,  39,  63, 298, 173, 102,
       100, 261, 103, 247,  61, 249, 129, 202, 252, 120,  35,  43, 170,
       230, 133, 273, 237, 263, 259, 142, 132, 182, 198,  74,  54, 292,
        17, 114, 118, 166,  55,  59, 191, 121, 144, 205, 254,  26, 294,
       215, 281, 151,  60,  46,  15, 176,   9, 211,  16,   1, 124, 110,
       289, 165,  84, 203,  79, 236, 128, 147, 113,  28,  88,  2